In [ ]:
from flask import Flask, redirect, url_for, render_template, request
from pyngrok import ngrok

from functions import (
    initialize_conversation,
    initialize_conv_reco,
    get_chat_model_completions,
    moderation_check,
    intent_confirmation_layer,
    compare_laptops_with_user,
    recommendation_validation,
    get_user_requirement_string,
    get_chat_completions_func_calling
)
import openai
import ast
import re
import pandas as pd
import json

openai.api_key=open("OpenAI_API_Key.txt",'r').read().strip()

app = Flask(__name__)

conversation_bot = []
conversation = initialize_conversation()
introduction = get_chat_model_completions(conversation)
conversation_bot.append({'bot':introduction})
top_3_laptops = None


@app.route("/")
def default_func():
    global conversation_bot, conversation, top_3_laptops
    return render_template("conversation_bot.html", name_xyz = conversation_bot)

@app.route("/end_conversation", methods = ['POST','GET'])
def end_conv():
    global conversation_bot, conversation, top_3_laptops
    conversation_bot = []
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)
    conversation_bot.append({'bot':introduction})
    top_3_laptops = None
    return redirect(url_for('default_func'))

@app.route("/conversation", methods = ['POST'])
def invite():
    global conversation_bot, conversation, top_3_laptops, conversation_reco
    user_input = request.form["user_input_message"]
    prompt = 'Remember your system message and that you are an intelligent laptop assistant. So, you only help with questions around laptop.'
    moderation = moderation_check(user_input)
    if moderation == 'Flagged':
        return redirect(url_for('end_conv'))

    if top_3_laptops is None:
        conversation.append({"role": "user", "content": user_input + prompt})
        conversation_bot.append({'user':user_input})

        response_assistant = get_chat_model_completions(conversation)


        moderation = moderation_check(response_assistant)
        if moderation == 'Flagged':
            return redirect(url_for('end_conv'))

        confirmation = intent_confirmation_layer(response_assistant)

        print('Intent confirmation is' + confirmation)

        moderation = moderation_check(confirmation)
        if moderation == 'Flagged':
            return redirect(url_for('end_conv'))

        if "No" in confirmation:
            conversation.append({"role": "assistant", "content": response_assistant})
            conversation_bot.append({'bot':response_assistant})
        else:
            response = get_user_requirement_string(response_assistant)
            result = get_chat_completions_func_calling(response, True)
            conversation_bot.append({'bot':"Thank you for providing all the information. Kindly wait, while I fetch the products: \n"})

            top_3_laptops = compare_laptops_with_user(result)

            validated_reco = recommendation_validation(top_3_laptops)

            if len(validated_reco) == 0:
                conversation_bot.append({'bot':"Sorry, we do not have laptops that match your requirements. Connecting you to a human expert. Please end this conversation."})

            conversation_reco = initialize_conv_reco(validated_reco)
            recommendation = get_chat_model_completions(conversation_reco)

            moderation = moderation_check(recommendation)
            if moderation == 'Flagged':
                return redirect(url_for('end_conv'))

            conversation_reco.append({"role": "user", "content": "This is my user profile" + response})

            conversation_reco.append({"role": "assistant", "content": recommendation})
            conversation_bot.append({'bot':recommendation})


    else:
        conversation_reco.append({"role": "user", "content": user_input})
        conversation_bot.append({'user':user_input})

        response_asst_reco = get_chat_model_completions(conversation_reco)

        moderation = moderation_check(response_asst_reco)
        if moderation == 'Flagged':
            return redirect(url_for('end_conv'))

        conversation.append({"role": "assistant", "content": response_asst_reco})
        conversation_bot.append({'bot':response_asst_reco})
    return redirect(url_for('default_func'))

if __name__ == "__main__":
    public_url = ngrok.connect(5001)
    print("ngrok tunnel:", public_url)
    app.run(port=5001)

ngrok tunnel: NgrokTunnel: "https://4a43-104-196-130-33.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:09:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:09:41] "GET /favicon.ico HTTP/1.1" 404 -


Intent confirmation isNo


INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:10:16] "POST /conversation HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:10:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:10:57] "POST /conversation HTTP/1.1" 302 -


Intent confirmation isNo
Intent confirmation isNo


...

INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:13:20] "POST /conversation HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:13:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:13:41] "POST /conversation HTTP/1.1" 302 -


Intent confirmation isNo


INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:13:42] "GET / HTTP/1.1" 200 -


Intent confirmation isYes


INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:14:57] "POST /conversation HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2024 16:14:58] "GET / HTTP/1.1" 200 -


In [9]:
!ngrok config add-authtoken 2o6BVasxITWe6GcAYwIboeIgsic_3AmpkFJ4sdhQ2F27aMANA


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
